In [2]:
!pip install streamlit # Install streamlit if you haven't already

import streamlit as st
import pandas as pd
import requests
import openai
from google.oauth2 import service_account
from googleapiclient.discovery import build

# ... (rest of your code) ...

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.1 MB/s eta 0:00:00


In [5]:
!pip install streamlit # Install streamlit if you haven't already

import streamlit as st
import pandas as pd
import requests
import openai

# Streamlit UI
st.title("AI Data Processing Agent")
st.write("Upload a CSV file to extract specific information using AI.")

# Step 1: File Upload
file = st.file_uploader("Upload CSV", type=["csv"])

# Step 2: Load Data
data = None

if file:
    data = pd.read_csv(file)
    st.write("Data Preview:", data.head())

# Continue only if data is loaded
if data is not None:
    # Step 3: User Prompt and Column Selection
    prompt_template = st.text_input("Enter search prompt (use {entry} for placeholder)",
                                    "Get the contact email of {entry}")
    primary_column = st.selectbox("Select the main column for search queries", data.columns)

    # Step 4: API Keys Setup
    serpapi_key = st.text_input("Enter SerpAPI Key", type="password")
    openai.api_key = st.text_input("Enter OpenAI API Key", type="password")

    # Step 5: Define Functions for Web Search and AI Processing
    def search_query(query):
        try:
            url = f"https://serpapi.com/search.json?q={query}&api_key={serpapi_key}"
            response = requests.get(url)
            return response.json()  # Adjust based on the API's response format
        except Exception as e:
            st.error(f"Error during web search: {e}")
            return None

    def extract_info(search_results, prompt):
        try:
            response = openai.Completion.create(
                engine="text-davinci-003",
                prompt=prompt + " " + str(search_results),
                max_tokens=50
            )
            return response.choices[0].text.strip()
        except Exception as e:
            st.error(f"Error with OpenAI API: {e}")
            return None

    # Step 6: Processing Data
    if st.button("Run Search and Extract Info"):
        results = []

        for index, entry in enumerate(data[primary_column]):
            st.write(f"Processing entry {index + 1}/{len(data)}: {entry}")

            # Create search query
            query = prompt_template.format(entry=entry)

            # Fetch search results
            search_results = search_query(query)

            if search_results:
                # Extract information using LLM
                extracted_info = extract_info(search_results, query)
                results.append(extracted_info)
            else:
                results.append(None)

        # Save results to DataFrame
        data['Extracted Info'] = results
        st.write("Results:", data)

        # Step 7: Download and Save Options
        st.download_button("Download CSV", data.to_csv(index=False).encode(), "results.csv", "text/csv")


2024-11-15 08:39:04.575 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-15 08:39:04.578 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-15 08:39:04.580 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-15 08:39:04.584 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-15 08:39:04.586 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-15 08:39:04.587 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-15 08:39:04.588 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-15 08:39:04.590 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [6]:
from google.colab import files
uploaded = files.upload()  # This will open a file dialog for you to upload the file


Saving contacts_extended.csv to contacts_extended.csv


In [10]:
import pandas as pd

# Replace 'yourfile.csv' with the name of the uploaded file
df = pd.read_csv('contacts_extended.csv')

# Show the first few rows of the data
df.head()

,Name,Company,Job Title,Email,Location
0,John Doe,ABC Corp,Software Engineer,john.doe@abccorp.com,New York
1,Jane Smith,XYZ Ltd,Data Scientist,jane.smith@xyz.com,San Francisco
2,Michael Brown,Tech Innovations,Product Manager,mike.brown@tech.com,Austin
3,Sarah Johnson,Web Solutions,UX Designer,sarah.j@websol.com,Chicago
4,David Lee,DataWorks,ML Engineer,david.lee@dataworks.com,Boston


In [8]:
# prompt: Using dataframe df: Austin

# Filter the DataFrame to only include rows where the Location is Austin
austin_df = df[df['Location'] == 'Austin']

# Display the filtered DataFrame
austin_df

,Name,Company,Job Title,Email,Location
2,Michael Brown,Tech Innovations,Product Manager,mike.brown@tech.com,Austin
